<a href="https://colab.research.google.com/github/ThisGuy245/AI-Tasks/blob/main/AI_Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

# Load the model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_response(prompt, max_new_tokens=50, temperature=0.7, top_k=50):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    start_time = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        no_repeat_ngram_size=2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    end_time = time.time()
    response_time = end_time - start_time
    return tokenizer.decode(outputs[0], skip_special_tokens=True), response_time

def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss)

# Main conversation loop
context = "You are Joe a regular human being. Try respond like a generic joe!"
max_context_length = 1000
conversation_history = ""

print("Welcome to your Joe Chat!")
print("You can start chatting with Joe. Type 'exit' to end the conversation.")

while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        break

    prompt = f"{context}\n{conversation_history}\nPlayer: {user_input}\nJoe:"
    response, response_time = generate_response(prompt, max_new_tokens=100, temperature=0.8, top_k=40)
    ai_response = response.split("Wizard:")[-1].strip()

    print(f"Joe: {ai_response}")
    print(f"Response time: {response_time:.2f} seconds")

    conversation_history += f"\nUser: {user_input}\nJoe: {ai_response}"
    if len(conversation_history) > max_context_length:
        conversation_history = conversation_history[-max_context_length:]

    # Calculate perplexity for the last exchange
    last_exchange = f"User: {user_input}\nJoe: {ai_response}"
    perplexity = calculate_perplexity(model, tokenizer, last_exchange)
    print(f"Perplexity of last exchange: {perplexity.item():.2f}")

print("Thank you for chatting! Here's a summary of our conversation:")
print(conversation_history)

# Final evaluation
final_perplexity = calculate_perplexity(model, tokenizer, conversation_history)
print(f"\nOverall conversation perplexity: {final_perplexity.item():.2f}")


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, I’m a language model but what I do in that role is to make everything
Hello, I’m a language model.” You’ll know the real name
Hello, I’m a language model’s­self, and I’m
Hello, I’m a language model, and I really wanted to make a nice API,
Hello, I’m a language model. I’think in any language. I�
